In [58]:
import pickle
import networkx as nx
import pandas as pd

from sklearn.metrics import f1_score
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [5]:
def clean_dict(pairs, use_lemma, reverse):
    new_pairs = {}
    for key, val in pairs.items():
        if use_lemma:
            term = key[0].split("(")[0].strip()
        else:
            term = key[0]
        target = key[1].split(",")[0]
        new_key = (target, term) if reverse else (term, target)
        new_pairs[new_key] = val

    return new_pairs


data = 'science'
in_name = '/home/LLM_Taxonomy/TExEval-2_testdata_1.2/data/sci/pred_hypernym/unified_lemmas_ppl.pickle'

path = "../gs_taxo/EN/" + str(data) + "_eurovoc_en.taxo"
G = nx.DiGraph()

with open(path, "r") as f:
    for line in f:
        idx, hypo, hyper = line.split("\t")
        hyper = hyper.replace("\n", "")
        G.add_node(hypo)
        G.add_node(hyper)
        G.add_edge(hyper, hypo)

with open(in_name, "rb") as f:
    ppls = pickle.load(f)

ppls_pairs = clean_dict(ppls, use_lemma=True, reverse=True)

In [80]:
df = {'forward':[], 'label': [], 'backward':[]}

for key, val in ppls_pairs.items():
    parent, child = key

    if child in G[parent]:
        df['label'].append(1)
    else:
        df['label'].append(0)
    

    forward_ppl = ppls_pairs[(child, parent)]
    backward_ppl = ppls_pairs[(parent, child)]
    df['forward'].append(forward_ppl)
    df['backward'].append(backward_ppl)


In [100]:
X = pd.DataFrame(df)
X['frac'] = X['backward'] / X['forward']
X['diff'] = X['backward'] - X['forward']
X['mul'] = X['backward'] * X['forward']
X['sign'] = X['frac'] < 0.5

scaler = preprocessing.Normalizer()
#X[['forward', 'backward', 'frac', 'diff', 'mul']] = scaler.fit_transform(X[['forward', 'backward', 'frac', 'diff', 'mul']] )

dropping = ['label', 'mul', 'sign']

logreg = LogisticRegression(C=0.01, max_iter=1000, fit_intercept=True, class_weight={0:1, 1:10000})
logreg.fit(X.drop(columns=dropping), X['label'])


LogisticRegression(C=0.01, class_weight={0: 1, 1: 10000}, max_iter=1000)

In [101]:

proba = logreg.predict_proba(X.drop(columns=dropping))
y_pred = proba[:,1] > 0.93

y_true = X['label']

pr = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc = roc_auc_score(y_true, y_pred)

pr, recall, f1, roc

(0.010974183682648197,
 0.9838709677419355,
 0.02170625389200249,
 0.6343977627471384)

In [83]:
X

,forward,label,backward,frac,diff,mul,sign
0,41.368931,0,65.500504,1.583326,24.131573,2709.685799,False
1,25.570189,0,172.101868,6.730567,146.531679,4400.677202,False
2,31.955431,0,93.412086,2.923199,61.456656,2985.023483,False
3,27.996063,0,30.926420,1.104670,2.930357,865.818016,False
4,38.091488,0,112.373703,2.950100,74.282215,4280.481546,False
...,...,...,...,...,...,...,...
15495,175.138718,0,30.362122,0.173360,-144.776596,5317.583039,True
15496,177.467621,0,26.909870,0.151633,-150.557751,4775.630632,True
15497,290.161804,0,27.617805,0.095181,-262.543999,8013.632266,True
15498,115.678574,0,16.913868,0.146214,-98.764706,1956.572119,True


In [162]:
pred = ((X['backward'] < 2) & (X['diff'] < -5) )

In [163]:
f1_score(X['label'], pred)

0.4040404040404041